# 1. Test Random Environment with OpenAI Gym

In [1]:
import pygame
import gym
import random

pygame 2.1.3 (SDL 2.0.22, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""print(tf.__version__)
print(len(tf.config.list_physical_devices('GPU'))>0)

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))"""

'print(tf.__version__)\nprint(len(tf.config.list_physical_devices(\'GPU\'))>0)\n\n# Check GPU availability\nprint("Num GPUs Available: ", len(tf.config.list_physical_devices(\'GPU\')))'

In [3]:
env = gym.make("SpaceInvaders-v0")
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:470.0
Episode:2 Score:125.0
Episode:3 Score:225.0
Episode:4 Score:80.0
Episode:5 Score:110.0


# 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(height, width, channels, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# 3. Build Agent with Keras-RL

In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [11]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr = 'eps', value_max = 1., value_min = .1, value_test = .2, nb_steps = 10000)
    memory = SequentialMemory(limit = 1000, window_length = 3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,
                   enable_dueling_network = True, dueling_type = 'avg',
                   nb_actions = actions, nb_steps_warmup = 1000
                   )
    
    return dqn

In [12]:
del model

model = build_model(height, width, channels, actions)

dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-4))

In [13]:
dqn.fit(env, nb_steps = 10000, visualize = False, verbose = 2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


  662/10000: episode: 1, duration: 78.649s, episode steps: 662, steps per second:   8, episode reward: 155.000, mean reward:  0.234 [ 0.000, 30.000], mean action: 2.414 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1155/10000: episode: 2, duration: 99.394s, episode steps: 493, steps per second:   5, episode reward: 110.000, mean reward:  0.223 [ 0.000, 30.000], mean action: 2.400 [0.000, 5.000],  loss: 30.946419, mean_q: 9.522295, mean_eps: 0.903025
 1874/10000: episode: 3, duration: 172.687s, episode steps: 719, steps per second:   4, episode reward: 180.000, mean reward:  0.250 [ 0.000, 30.000], mean action: 2.441 [0.000, 5.000],  loss: 1.476381, mean_q: 8.079699, mean_eps: 0.863740
 2832/10000: episode: 4, duration: 227.776s, episode steps: 958, steps per second:   4, episode reward: 185.000, mean reward:  0.193 [ 0.000, 30.000], mean action: 2.633 [0.000, 5.000],  loss: 0.983374, mean_q: 8.706827, mean_eps: 0.788275
 3564/10000: episode: 5, duration: 174.563s, episode steps:

In [14]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 10 episodes ...
Episode 1: reward: 45.000, steps: 466
Episode 2: reward: 60.000, steps: 508
Episode 3: reward: 20.000, steps: 433
Episode 4: reward: 80.000, steps: 527
Episode 5: reward: 385.000, steps: 1115
Episode 6: reward: 330.000, steps: 1137
Episode 7: reward: 130.000, steps: 542
Episode 8: reward: 40.000, steps: 570
Episode 9: reward: 395.000, steps: 1619
Episode 10: reward: 195.000, steps: 652
168.0


# 4. Reloading Agent from Memory

In [15]:
dqn.save_weights('Saved_Weights.h5f')

[TIP] Next time specify overwrite=True!


In [16]:
del model, dqn

In [17]:
model = build_model(height, width, channels, actions)

dqn = build_agent(model, actions)
dqn.compile(Adam(lr = 1e-4))

In [18]:
dqn.load_weights('Saved_Weights.h5f')

In [20]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))
env.close()

Testing for 10 episodes ...
Episode 1: reward: 480.000, steps: 1188
Episode 2: reward: 150.000, steps: 642
Episode 3: reward: 375.000, steps: 1232
Episode 4: reward: 70.000, steps: 570
Episode 5: reward: 355.000, steps: 956
Episode 6: reward: 185.000, steps: 788
Episode 7: reward: 75.000, steps: 561
Episode 8: reward: 235.000, steps: 993
Episode 9: reward: 135.000, steps: 642
Episode 10: reward: 25.000, steps: 540
208.5
